In [1]:
# # Install necessary packages
#!pip install -U accelerate==0.29.3 peft==0.10.0 bitsandbytes==0.43.1 trl==0.8.6 datasets==2.19.0


In [1]:
# Import necessary libraries
import os
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from trl import SFTTrainer
from datasets import load_dataset
!nvidia-smi

2024-12-04 21:43:20.756556: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-04 21:43:20.776788: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-04 21:43:20.776809: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-04 21:43:20.777407: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-04 21:43:20.781048: I tensorflow/core/platform/cpu_feature_guar

Wed Dec  4 21:43:22 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.28.03              Driver Version: 560.28.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               Off |   00000000:01:00.0 Off |                  Off |
| 53%   78C    P0            229W /  230W |   13301MiB /  24564MiB |    100%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
# Set quantization configuration
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Define the base model name 
base_model_name = "beomi/Llama-3-Open-Ko-8B"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    base_model_name,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Load the model with quantization configuration
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True,
)
model.config.use_cache = False

# Prepare the model for k-bit training
model = prepare_model_for_kbit_training(model)

# Set up LoRA configuration for PEFT
peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    lora_dropout=0.5,
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply PEFT to the model
model = get_peft_model(model, peft_config)


/home/billy/anaconda3/lib/python3.9/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [3]:

# Load and preprocess the dataset (kept from previous code)
dataset = load_dataset("doosup01/kpro_last-dataset", split="train")

def preprocess_function(examples):
    prompts = examples["prompt"]
    completions = examples["completion"]
    combined_texts = [f"{p}{c}" for p, c in zip(prompts, completions)]
    return {"text": combined_texts}

dataset = dataset.map(preprocess_function, batched=True)


In [4]:

# Set up training arguments
output_dir = "./2kpro_last_ds_results"
os.makedirs(output_dir, exist_ok=True)

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=True,
    logging_steps=50,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    report_to="tensorboard",
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_args,
    packing=False,
)

# Train the model
trainer.train()


/home/billy/anaconda3/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/billy/anaconda3/lib/python3.9/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/home/billy/anaconda3/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To mainta

/home/billy/anaconda3/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/billy/anaconda3/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/billy/anaconda3/lib/python3.9/site-packages/torch/utils/checkpoi

Step,Training Loss
50,1.779000
100,1.454300
150,1.361500
200,1.343200
250,1.297400
300,1.312300
350,1.311700
400,1.281900
450,1.237400
500,1.284400


/home/billy/anaconda3/lib/python3.9/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/billy/anaconda3/lib/python3.9/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/billy/anaconda3/lib/python3.9/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new d

TrainOutput(global_step=2950, training_loss=1.196121249441373, metrics={'train_runtime': 17656.2147, 'train_samples_per_second': 0.668, 'train_steps_per_second': 0.167, 'total_flos': 1.5250329944653824e+17, 'train_loss': 1.196121249441373, 'epoch': 5.0})

In [6]:
# After training, merge and save the model
model = trainer.model
model = model.merge_and_unload()
model.config.use_cache = True

model_save_path = "2ds_kpro_last_model"
model.save_pretrained(model_save_path, safe_serialization=True)
tokenizer.save_pretrained(model_save_path)

# Load the saved model for inference
model = AutoModelForCausalLM.from_pretrained(
    model_save_path,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=quant_config,
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_save_path, trust_remote_code=True)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/home/billy/anaconda3/lib/python3.9/site-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
# Set up the text generation pipeline
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=600,
    # truncation=True,
    no_repeat_ngram_size=5,
    repetition_penalty=1.2,  # 반복 페널티 추가
    eos_token_id=tokenizer.eos_token_id,
)

# Generate a response to a test prompt
test_prompt = "최근 들어 일상생활에서 불안감과 무기력함을 자주 느끼고 있습니다. 특히 직장에서 업무를 수행할 때 자신감이 떨어지고, 사소한 실수에도 과도하게 걱정하는 경향이 있습니다. 또한 가족이나 친구들과의 관계에서도 예전만큼 즐거움을 느끼지 못하고 있어요. 이런 감정들이 제 삶에 어떤 영향을 미치고 있는지, 그리고 이를 개선하기 위해 어떤 방법들을 시도해볼 수 있을지 조언을 구하고 싶습니다."
input_text = f"의료 상담 전문가가 되어 다음 질문에 답하여라. 문장은 습니다, 합니다, 등의 공손한 말투를 사용하여라.\n### 질문: {test_prompt}\n### 답변:"
result = pipe(
    input_text,
    num_return_sequences=1,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,  
)

# Display the generated text
print(result[0]['generated_text'])

의료 상담 전문가가 되어 다음 질문에 답하여라. 문장은 습니다, 합니다, 등의 공손한 말투를 사용하여라.
### 질문: 최근 들어 일상생활에서 불안감과 무기력함을 자주 느끼고 있습니다. 특히 직장에서 업무를 수행할 때 자신감이 떨어지고, 사소한 실수에도 과도하게 걱정하는 경향이 있습니다. 또한 가족이나 친구들과의 관계에서도 예전만큼 즐거움을 느끼지 못하고 있어요. 이런 감정들이 제 삶에 어떤 영향을 미치고 있는지, 그리고 이를 개선하기 위해 어떤 방법들을 시도해볼 수 있을지 조언을 구하고 싶습니다.
### 답변: 먼저 불안과 우울증 증세로 인해 여러분께서 겪으신 고통들에 대해 위로와 응원을 보내드립니다. 무엇보다 중요한 것은 정신적 상태의 악화는 단순히 마음 속으로부터 발생되는 문제가 아닙니다. 그보다는 육체적인 건강까지 함께 고려해야 하며 이러한 이유 때문에 치료 및 관리 측면에서의 접근 방식 역시 다양한 분야로부터 진행되어야 한다고 생각합니다.

어떠세요? 이건 잘 만든 것 같죠?
ㅋㅋㅋㅋㅋ
아니 #@기타#요...
네...ㅎ
다음은 또 언제 올려
